# This notebook provides portfolio weights based on the historical data to the backtesting notebook

In [1]:
# !pip uninstall deepdowmine
#!pip install git+https://github.com/dsman1823/deepdowmine.git
#!conda install pytorch torchvision torchaudio cudatoolkit=10.2 -c pytorch # <---- to Enable SVD 

In [2]:
import torch

import pandas as pd
import numpy as np

from datetime import datetime 
from deepdowmine.nn import LinearNetMine, UpdNumericalMarkowitzWithShorting, UpdLinearNetMine, UpdDenseNet, UpdUpdLinearNetMine
from deepdowmine.nn import MinVarDenseNet, ConvNetFullOpti, ConvNetMinVar

In [3]:
RETS_FILE_PATH = 'historical_returns.csv'



loockback, gap, horizon = 50, 0, 5
n_assets = 5
loockback = 50


## Load NN from dict

In [4]:
LINEAR_NET_OSHARPE_FILE_PATH = r'./NNs/linear_net_50x5_sharpe.pth' # LinearNetMine with ordinary Sharpe cost

In [5]:
# years parameter reprsenets amount of years after the backtest stasrt: the model dowloaded is the the model trained 
# on the period (train_start, back_test_start + years) 


# LinearNetMine with ordinary sharpe cost function
def load_linear_net_osharpe(years=0):
    network = LinearNetMine(1, loockback, n_assets, p=0.5)
    network.load_state_dict(torch.load(fr'./NNs/linear_net_50x5_sharpe_{years}.pth'))
    print(fr'./NNs/linear_net_50x5_sharpe_{years}.pth')
    return network.eval()
    

In [6]:
def load_dense_net(years=0):
    network = UpdLinearNetMine(1, loockback, n_assets, p=0.5)
    network.load_state_dict(torch.load(fr'./NNs/dense_{years}.pth'))
    print(fr'dense_{years}.pth')
    return network.eval()

## Core functions

In [7]:
def transform_rets_to_NN_input(rets):
    # tranform (loockback, n_assets) df into (1, 1, loockback, n_assets)=(sample_size, n_channels, loockback, n_assets) tensor 
    returns_np = rets.to_numpy()

    # Add the required dimensions: (n_samples, n_channels, lookback, n_assets)
    returns_np_expanded = np.expand_dims(returns_np, axis=0)  # Adds n_samples dimension
    returns_np_expanded = np.expand_dims(returns_np_expanded, axis=0)  
    return torch.from_numpy(returns_np_expanded).float()

def weights_from_NN(rets, network):
    X = transform_rets_to_NN_input(rets)
    return network(X).detach().numpy()[0]

# method is a function, used for obtaining the returns
def get_weights(method):
    rets = pd.read_csv(RETS_FILE_PATH, index_col=0)
    return method(rets)

In [8]:
rets = pd.read_csv(RETS_FILE_PATH, index_col=0)
X = transform_rets_to_NN_input(rets)
network = LinearNetMine(1, loockback, n_assets, p=0.5)
network.load_state_dict(torch.load(fr'./NNs/linear_net_50x5_sharpe_{2}.pth'))
network.eval()(X).detach().numpy()[0]

array([ 0.06418574,  0.25817055, -0.01574563,  0.60880786,  0.08458146],
      dtype=float32)

## NNs and related methods


### NNs

networks[i] method is trained on (train_start, backtest_start + i years) data interval

In [9]:
years = [0, 1, 2]#, 1, 2]

linear_osharpe_networks = [load_linear_net_osharpe(years=i) for i in years]
dense_networks = [load_dense_net(years=i) for i in years]

./NNs/linear_net_50x5_sharpe_0.pth
./NNs/linear_net_50x5_sharpe_1.pth
./NNs/linear_net_50x5_sharpe_2.pth
dense_0.pth
dense_1.pth
dense_2.pth


### methods

In [10]:
dense_networks[1].linear0.bias[0:3]

tensor([ 0.0484,  0.0029, -0.0442], grad_fn=<SliceBackward0>)

In [11]:
dense_networks[2].linear0.bias[0:3]

tensor([ 0.0484,  0.0029, -0.0442], grad_fn=<SliceBackward0>)

In [12]:
# note the format of lambda 
linear_osharpe_methods = [lambda r, i=i: weights_from_NN(r, linear_osharpe_networks[i]) for i in years] 
dense_methods = [lambda r, i=i: weights_from_NN(r, dense_networks[i]) for i in years] 


In [13]:
[get_weights(dense_methods[i]) for i in years]

[array([0.01819081, 0.24908274, 0.29753768, 0.41258937, 0.0225994 ],
       dtype=float32),
 array([-0.03791929,  0.1064674 ,  0.4017897 ,  0.46670547,  0.06295667],
       dtype=float32),
 array([-0.03175861,  0.18791872,  0.14897145,  0.48070765,  0.21416074],
       dtype=float32)]

# Server

In [14]:
from dateutil.relativedelta import relativedelta
from datetime import datetime, date

RETRAINING_DATE1 = date(2021, 12, 15)
RETRAINING_DATE2 = date(2022, 12, 15)

In [15]:
tmp = True
not tmp

False

In [21]:
from flask import Flask, request, jsonify
from flask_cors import CORS 

import numpy as np

app = Flask(__name__)
CORS(app)  # Enable CORS for your Flask app

markers = {}
markers['net_initialized'] = False

@app.route('/get_weights', methods=['GET'])
def get_request():
    # Extract date string from the query parameters
    date_str = request.args.get('date', None)
    
    # Simple validation to check if date is provided
    if not date_str:
        return jsonify({"error": "Missing date parameter"}), 400

    # Try to convert the date string to a datetime object
    try:
        # Note that now we're only parsing the date, not the time
        date = datetime.strptime(date_str, "%Y-%m-%d").date()
    except ValueError:
        # If there is an error in parsing the date, return an error message
        return jsonify({"error": "Invalid date format. Please use YYYY-MM-DD format."}), 400
    
#     years_plus = 0
#     if RETRAINING_DATE1 < date < RETRAINING_DATE2:
#         years_plus = 1
#         print(f'Working with model: {years_plus}')
#     if RETRAINING_DATE2 < date:
#         years_plus = 2
#         print(f'Working with model: {years_plus}')
    
#     weights = get_weights(linear_osharpe_methods[years_plus]).tolist()
    
    rets = pd.read_csv(RETS_FILE_PATH, index_col=0)
    X = transform_rets_to_NN_input(rets)
    
    network = MinVarDenseNet(1, 50, 5)
    network.load_state_dict(torch.load(fr'./NNs/min_var_dense_net_0.pth'))
       
#     if date < RETRAINING_DATE1:
#         print(0)
#     if RETRAINING_DATE1 < date < RETRAINING_DATE2:
#         print('1')
#         network = MinVarDenseNet(1, 50, 5)
#         network.load_state_dict(torch.load(fr'./NNs/min_var_dense_net_1.pth'))
#     if RETRAINING_DATE2 < date:
#         network = MinVarDenseNet(1, 50, 5)
#         network.load_state_dict(torch.load(fr'./NNs/min_var_dense_net_2.pth'))
 #       print('2')
    
    weights = network.eval()(X).detach().numpy()[0].tolist()
    
        
    #weights = weights_from_NN(rets, linear_osharpe_networks[0]).tolist()
    #print(weights)
    # Return the vector as JSON, using the string representation of the date for simplicity
    return jsonify({'weights': weights})


In [22]:
rets = pd.read_csv(RETS_FILE_PATH, index_col=0)
X = transform_rets_to_NN_input(rets)
network = LinearNetMine(1, loockback, n_assets, p=0.5)
network.load_state_dict(torch.load(fr'./NNs/linear_net_50x5_sharpe_{0}.pth'))
weights = network.eval()(X).detach().numpy()[0].tolist()
weights, sum(weights)

([0.15217816829681396,
  0.7109567523002625,
  -0.0016196349170058966,
  0.1990373432636261,
  -0.06055258587002754],
 1.000000043073669)

In [ ]:
app.run(port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [13/Mar/2024 11:59:32] "GET /get_weights?date=2020-12-21 HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2024 11:59:34] "GET /get_weights?date=2020-12-28 HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2024 11:59:36] "GET /get_weights?date=2021-01-04 HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2024 11:59:38] "GET /get_weights?date=2021-01-11 HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2024 11:59:40] "GET /get_weights?date=2021-01-19 HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2024 11:59:42] "GET /get_weights?date=2021-01-25 HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2024 11:59:45] "GET /get_weights?date=2021-02-01 HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2024 11:59:47] "GET /get_weights?date=2021-02-08 HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2024 11:59:49] "GET /get_weights?date=2021-02-16 HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2024 11:59:51] "GET /get_weights?date=2021-02-22 HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2024 11:59:53] "GET /get_weights?date=2021-03-01 HT

127.0.0.1 - - [13/Mar/2024 12:02:51] "GET /get_weights?date=2022-10-03 HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2024 12:02:54] "GET /get_weights?date=2022-10-10 HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2024 12:02:56] "GET /get_weights?date=2022-10-17 HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2024 12:02:58] "GET /get_weights?date=2022-10-24 HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2024 12:03:00] "GET /get_weights?date=2022-10-31 HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2024 12:03:02] "GET /get_weights?date=2022-11-07 HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2024 12:03:04] "GET /get_weights?date=2022-11-14 HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2024 12:03:07] "GET /get_weights?date=2022-11-21 HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2024 12:03:09] "GET /get_weights?date=2022-11-28 HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2024 12:03:11] "GET /get_weights?date=2022-12-05 HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2024 12:03:13] "GET /get_weights?date=2022-12-12 HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2024 12:03:15] "GET /

127.0.0.1 - - [13/Mar/2024 12:11:15] "GET /get_weights?date=2021-08-02 HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2024 12:11:17] "GET /get_weights?date=2021-08-09 HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2024 12:11:19] "GET /get_weights?date=2021-08-16 HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2024 12:11:21] "GET /get_weights?date=2021-08-23 HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2024 12:11:23] "GET /get_weights?date=2021-08-30 HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2024 12:11:25] "GET /get_weights?date=2021-09-07 HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2024 12:11:27] "GET /get_weights?date=2021-09-13 HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2024 12:11:30] "GET /get_weights?date=2021-09-20 HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2024 12:11:32] "GET /get_weights?date=2021-09-27 HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2024 12:11:34] "GET /get_weights?date=2021-10-04 HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2024 12:11:36] "GET /get_weights?date=2021-10-11 HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2024 12:11:38] "GET /

127.0.0.1 - - [13/Mar/2024 12:14:38] "GET /get_weights?date=2023-05-30 HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2024 12:14:40] "GET /get_weights?date=2023-06-05 HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2024 12:14:42] "GET /get_weights?date=2023-06-12 HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2024 12:14:44] "GET /get_weights?date=2023-06-19 HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2024 12:14:47] "GET /get_weights?date=2023-06-26 HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2024 12:14:49] "GET /get_weights?date=2023-07-03 HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2024 12:14:51] "GET /get_weights?date=2023-07-10 HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2024 12:14:53] "GET /get_weights?date=2023-07-17 HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2024 12:14:55] "GET /get_weights?date=2023-07-24 HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2024 12:14:57] "GET /get_weights?date=2023-07-31 HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2024 12:14:59] "GET /get_weights?date=2023-08-07 HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2024 12:15:02] "GET /

127.0.0.1 - - [13/Mar/2024 12:24:30] "GET /get_weights?date=2022-03-28 HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2024 12:24:32] "GET /get_weights?date=2022-04-04 HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2024 12:24:34] "GET /get_weights?date=2022-04-11 HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2024 12:24:36] "GET /get_weights?date=2022-04-18 HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2024 12:24:38] "GET /get_weights?date=2022-04-25 HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2024 12:24:41] "GET /get_weights?date=2022-05-02 HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2024 12:24:43] "GET /get_weights?date=2022-05-09 HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2024 12:24:45] "GET /get_weights?date=2022-05-16 HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2024 12:24:47] "GET /get_weights?date=2022-05-23 HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2024 12:24:49] "GET /get_weights?date=2022-05-31 HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2024 12:24:51] "GET /get_weights?date=2022-06-06 HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2024 12:24:54] "GET /

In [ ]:
update the 